In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from extract import POLOCM2
from utils import read_plan
from evaluator import ExecutabilityEvaluator

ImportError: DLL load failed while importing _multiarray_umath: 找不到指定的模块。

ImportError: DLL load failed while importing _multiarray_umath: 找不到指定的模块。

In [2]:
def test_exe(domain_name, trace, test_trace=None):
    debug = {
        # 'get_transition_sets': True,
        # 'extract_holes': True,
        # 'find_holes': True,
    }
    try:
        polocm2 = POLOCM2(state_param=True, viz=False, debug=debug)
        model = polocm2.extract_model([trace])
        
        actions = model.actions
        
        pddl_domain = model.to_pddl_domain("test_bw")
        evaluator = ExecutabilityEvaluator(pddl_domain, 'overall', debug=True)
        if test_trace is not None:
            action_trace = [step.action for step in test_trace]
            exe = evaluator.get_overall_executability('l', action_trace, set(), set())
        else:
            action_trace = [step.action for step in trace]
            exe = evaluator.get_overall_executability('l', action_trace, set(), set())
        if exe < 1:
            print(f"Warning! domain: {domain_name} - executability: {exe}")
        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0

In [ ]:
def test_cross_exe(domain_name, po_trace, to_trace):
    debug = {}
    try:
        polocm2 = POLOCM2(state_param=True, viz=False, debug=debug)
        model = polocm2.extract_model([po_trace])
        learned_domain = model.to_pddl_domain(domain_name)
        gt_filename = os.path.join('../../data', 'goose-benchmarks', 'tasks', domain_name, 'domain.pddl')
        evaluator = ExecutabilityEvaluator(learned_domain, 'cross',gt_filename, debug=False)
        action_trace = [step.action for step in to_trace]
        exe = evaluator.get_cross_executabilities(action_trace)
        return exe
    except Exception as e:
        print(f"Error processing domain {domain_name}: {e}")
        return 0

In [4]:
test_plan = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
test_trace = read_plan(test_plan)
po_trace = test_trace.to_partial_ordered_trace(0.3)

# test_exe("plan1", trace)
test_exe("plan2", po_trace)
# plan2 = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
# test_exe("test_bw", read_plan(plan2))

action <PropositionalAction '(stack zero b3 b2)' at 0x16a1e14ea50> not executable
preconditions not satisfied:  {<Atom s0f0_state1(zero)>}
Warning! domain: plan2 - executability: 0.75


0.75

In [5]:
test_plan = "(pickup b2?object),(stack b2?object b1?object),(pickup b3?object),(stack b3?object b2?object)"
test_trace = read_plan(test_plan)
po_trace = test_trace.to_partial_ordered_trace(0.3)


test_cross_exe("blocksworld", po_trace, test_trace)

action (pickup b2) executed
true_effs:  {<Atom holding(b2)>}
action (stack b2 b1) executed
true_effs:  {<Atom clear(b2)>, <Atom arm-empty()>, <Atom on(b2, b1)>}
action (pickup b3) executed
true_effs:  {<Atom holding(b3)>, <Atom clear(b2)>, <Atom on(b2, b1)>}
action (stack b3 b2) executed
true_effs:  {<Atom clear(b3)>, <Atom arm-empty()>, <Atom on(b2, b1)>, <Atom on(b3, b2)>}
type_objs:  defaultdict(<class 'set'>, {'object': {'b1', 'b2', 'b3'}})
init_effs:  {<Atom clear(b3)>, <Atom arm-empty()>, <Atom on(b2, b1)>, <Atom on(b3, b2)>}
learned model seqs:  [[pickup <TypedObject b3: na>, stack <TypedObject b1: na> <TypedObject b3: na>, pickup <TypedObject b1: na>], [pickup <TypedObject b3: na>, stack <TypedObject b1: na> <TypedObject b3: na>, pickup <TypedObject b1: na>], [pickup <TypedObject b1: na>, stack <TypedObject b1: na> <TypedObject b1: na>, pickup <TypedObject b1: na>], [pickup <TypedObject b1: na>, stack <TypedObject b1: na> <TypedObject b1: na>, pickup <TypedObject b3: na>], [pic

(0.05, 0.2666666666666667)

In [6]:
# with open("../../data/plain_traces/plain_traces.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         details = line.split("&&")
#         name = f"{details[0]}-{details[2]}-{details[3]}"
      
#         plan = details[-1]
#         trace = read_plan(plan)
#         po_trace = trace.to_partial_ordered_trace(0.3)
#         test_exe(name, po_trace)